In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sb
import plotly
import sqlalchemy
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Float, text, select, Text
from sqlalchemy.orm import sessionmaker


sb.set_context('talk')

#estoy probando si anda

**Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones**

**Exploración y Curación de Datos**

*Edición 2024*

----

# Trabajo práctico entregable - parte 1

En esta notebook, vamos a cargar el conjunto de datos de [la compentencia Kaggle](https://www.kaggle.com/dansbecker/melbourne-housing-snapshot) sobre estimación de precios de ventas de propiedades en Melbourne, Australia.

Utilizaremos el conjunto de datos reducido producido por [DanB](https://www.kaggle.com/dansbecker). Hemos subido una copia a un servidor de la Universidad Nacional de Córdoba para facilitar su acceso remoto.

In [2]:
melb_df = pd.read_csv(
    './ArchivosCSV/melb_data.csv')

melb_df[:5]

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,...,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


In [3]:
interesting_cols = [
  'description', 'neighborhood_overview',
  'street', 'neighborhood', 'city', 'suburb', 'state', 'zipcode',
  'price', 'weekly_price', 'monthly_price',
  'latitude', 'longitude',
]

airbnb_df = pd.read_csv(
  './ArchivosCSV/cleansed_listings_dec18.csv',
  usecols = interesting_cols
)

C:\Users\juanc\AppData\Local\Temp\ipykernel_11080\379606573.py:8: DtypeWarning: Columns (35) have mixed types. Specify dtype option on import or set low_memory=False.
  airbnb_df = pd.read_csv(


## Ejercicio 1 SQL:

1. Crear una base de datos en SQLite utilizando la libreria SQLalchemy.

2. Ingestar los datos provistos en 'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/melb_data.csv' en una tabla y el dataset generado en clase con datos de airbnb y sus precios por codigo postal en otra.

3. Implementar consultas en SQL que respondan con la siguiente información:

    - cantidad de registros totales por ciudad.
    - cantidad de registros totales por barrio y ciudad.

4. Combinar los datasets de ambas tablas ingestadas utilizando el comando JOIN de SQL  para obtener un resultado similar a lo realizado con Pandas en clase.  


In [4]:
#Se crea la BDD
engine = create_engine('sqlite:///melb_data.db') # Crea una conexion a la BDD
metadata = MetaData() # Se utiliza para manejar la estructura de la BDD

##### BDD de melb_data.csv

In [5]:
melb_df['postcode_int'] = pd.to_numeric(melb_df.Postcode, errors='coerce').fillna(0).astype(int)
#Se guarda el archivo CSV en un tabla dentro de la BDD
melb_df.to_sql(
    'melb_data', #Nombre de la tabla
    con = engine, #Especifica a qué BDD se va a conectar
    if_exists='replace', 
    index=False
)

13580

In [6]:
query_por_region = """
SELECT Regionname, COUNT(*) AS registros_totales
FROM melb_data
GROUP BY Regionname;
"""


df_total_por_region = pd.read_sql_query(
    query_por_region, 
    con=engine
)


print("Cantidad de registros totales por Región:")
print(df_total_por_region)


Cantidad de registros totales por Región:
                   Regionname  registros_totales
0        Eastern Metropolitan               1471
1            Eastern Victoria                 53
2       Northern Metropolitan               3890
3           Northern Victoria                 41
4  South-Eastern Metropolitan                450
5       Southern Metropolitan               4695
6        Western Metropolitan               2948
7            Western Victoria                 32


In [7]:
query_por_suburbio = """
SELECT Suburb, COUNT(*) AS registros_totales
FROM melb_data
GROUP BY Suburb;
"""


df_total_por_suburbio = pd.read_sql_query(
    query_por_suburbio, 
    con=engine
)


print("Cantidad de registros totales por Suburbio:")
print(df_total_por_suburbio)


Cantidad de registros totales por Suburbio:
           Suburb  registros_totales
0      Abbotsford                 56
1      Aberfeldie                 44
2    Airport West                 67
3       Albanvale                  6
4     Albert Park                 69
..            ...                ...
309    Wonga Park                  1
310  Wyndham Vale                  4
311     Yallambie                 24
312    Yarra Glen                  1
313    Yarraville                164

[314 rows x 2 columns]


##### BDD de AirBNB

In [8]:
airbnb_df['zipcode_int'] = pd.to_numeric(airbnb_df.zipcode, errors='coerce').fillna(0).astype(int)
airbnb_df.to_sql(
    'airbnb_data', #Nombre de la tabla
    con=engine, #Especifica a qué BDD se va a conectar
    if_exists='replace', #Si ya existe la tabla, se reemplaza con los datos nuevos
    index=False
)

22895

In [9]:
query_por_ciudad_airbnb = """
SELECT city, COUNT(*) AS registros_totales
FROM airbnb_data
GROUP BY city;
"""


df_total_por_ciudad = pd.read_sql_query(
    query_por_ciudad_airbnb, 
    con=engine
)


print("Cantidad de registros totales por ciudad:")
print(df_total_por_ciudad)

Cantidad de registros totales por ciudad:
                 city  registros_totales
0             Banyule                203
1             Bayside                375
2          Boroondara                664
3            Brimbank                108
4            Cardinia                123
5               Casey                153
6             Darebin                698
7           Frankston                177
8           Glen Eira                631
9   Greater Dandenong                147
10        Hobsons Bay                239
11               Hume                170
12           Kingston                309
13               Knox                175
14         Manningham                313
15        Maribyrnong                436
16          Maroondah                115
17          Melbourne               7368
18             Melton                 95
19             Monash                571
20      Moonee Valley                344
21           Moreland                967
22          Nil

In [10]:
query_por_barrio_ciudad_airbnb = """
SELECT city, neighborhood, COUNT(*) AS registros_totales
FROM airbnb_data
GROUP BY city, neighborhood;
"""


df_por_barrio_ciudad = pd.read_sql_query(
    query_por_barrio_ciudad_airbnb, 
    con=engine
)


print("\nCantidad de registros totales por barrio y ciudad:")
print(df_por_barrio_ciudad)


Cantidad de registros totales por barrio y ciudad:
             city   neighborhood  registros_totales
0         Banyule           None                174
1         Banyule     Alphington                 16
2         Banyule        Preston                  8
3         Banyule      Thornbury                  5
4         Bayside           None                170
..            ...            ...                ...
129         Yarra        Fitzroy                381
130         Yarra  Fitzroy North                225
131         Yarra      Northcote                  6
132         Yarra       Richmond                547
133  Yarra Ranges           None                771

[134 rows x 3 columns]


##### JOIN

In [11]:
print("Suburbio")
print("")
print("Tipo de datos de la columna Suburb en melb_data:", melb_df.Suburb.dtype)
print("Tipo de datos de la columna suburb en airbnb_data:", airbnb_df.suburb.dtype)
print("")
print("")
print("")
print("Codigo postal")
print("")
print("Tipo de datos de la columna postcode en melb_data:", melb_df.postcode_int.dtype)
print("Tipo de datos de la columna zipcode en airbnb_data:", airbnb_df.zipcode_int.dtype)

Suburbio

Tipo de datos de la columna Suburb en melb_data: object
Tipo de datos de la columna suburb en airbnb_data: object



Codigo postal

Tipo de datos de la columna postcode en melb_data: int32
Tipo de datos de la columna zipcode en airbnb_data: int32


In [88]:
#Analisis de conveniencia por Suburbio
melb_suburbs = melb_df.Suburb.astype(str).values
airbnb_suburbs = airbnb_df.suburb.astype(str).values

intersection = np.intersect1d(
    melb_suburbs,
    airbnb_suburbs, 
    assume_unique=False
)

print("Suburbios totales en melb_data:", len(melb_df.Suburb.unique()))
print("Suburbios totales en airbnb_data:", len(airbnb_df.suburb.unique()))
print("Suburbios en común:", len(intersection))

Suburbios totales en melb_data: 314
Suburbios totales en airbnb_data: 543
Suburbios en común: 296


In [89]:
#Analisis de conveniencia por Codigo postal
intersection = np.intersect1d(
    melb_df.postcode_int,
    airbnb_df.zipcode_int, 
    assume_unique=False
)

print("CP totales en melb_data:", len(melb_df.postcode_int.unique()))
print("CP totales en airbnb_data:", len(airbnb_df.zipcode_int.unique()))
print("CP en común:", len(intersection))

CP totales en melb_data: 198
CP totales en airbnb_data: 248
CP en común: 191


Luego de este analisis se puede concluir que es mas conveniente tomar las columnas referentes al codigo postal ya que de lo contrario se podrian relacionar menos columnas entre si

In [12]:
query_precios="""
SELECT zipcode_int CodigoPostal, 
COUNT(*) Cantidad,
 
AVG(price) PrecioDiario,
AVG(weekly_price) PrecioSemanal, 
AVG(monthly_price) PrecioMensual

FROM airbnb_data a

WHERE a.zipcode_int > 0
GROUP BY zipcode_int
ORDER BY 2 desc
"""


with engine.connect() as con:
      rs = con.execute(text(query_precios))
      df_precios=pd.DataFrame(rs.fetchall())

print(df_precios)

     CodigoPostal  Cantidad  PrecioDiario  PrecioSemanal  PrecioMensual
0            3000      3367    150.504307     918.738956    3407.204651
1            3006      1268    188.851735    1239.827273    4358.556701
2            3182      1135    147.729515    1051.441026    3847.400000
3            3141       881    157.912599    1040.403670    3417.811111
4            3004       728    158.271978    1061.171429    3759.925926
..            ...       ...           ...            ...            ...
242          3200         1     40.000000            NaN            NaN
243          3096         1    100.000000     250.000000    1500.000000
244          2582         1    104.000000            NaN            NaN
245          2134         1     50.000000            NaN            NaN
246          2010         1     40.000000            NaN            NaN

[247 rows x 5 columns]


In [13]:
#Guardamos df_precios en SQLite
df_precios.to_sql(
      'precios_data', 
      con=engine, 
      if_exists="replace"
)

247

In [104]:
#Combinamos las tablas precios_data con melb_data
query_precios_join_melb="""
SELECT 

a.postcode_int, 
a.Suburb, 
a.Price, 
a.Distance,
a.Regionname,
Bedroom2,
Bathroom,
BuildingArea,
b.PrecioDiario,
b.PrecioSemanal,
b.PrecioMensual, 
b.Cantidad

FROM melb_data a LEFT JOIN precios_data b

ON b.CodigoPostal=a.postcode_int
ORDER BY Cantidad desc 
"""


with engine.connect() as con:
      precios_join_melb = con.execute(text(query_precios_join_melb))
      df_precios_join = pd.DataFrame( precios_join_melb.fetchall())

df_precios_join

,postcode_int,Suburb,Price,Distance,Regionname,Bedroom2,Bathroom,BuildingArea,PrecioDiario,PrecioSemanal,PrecioMensual,Cantidad
0,3000,Melbourne,427000.0,2.8,Northern Metropolitan,1.0,1.0,NaN,150.504307,918.738956,3407.204651,3367.0
1,3000,Melbourne,1160500.0,2.8,Northern Metropolitan,2.0,2.0,NaN,150.504307,918.738956,3407.204651,3367.0
2,3000,Melbourne,780000.0,2.8,Northern Metropolitan,2.0,2.0,85.0,150.504307,918.738956,3407.204651,3367.0
3,3000,Melbourne,535000.0,2.8,Northern Metropolitan,2.0,1.0,84.0,150.504307,918.738956,3407.204651,3367.0
4,3000,Melbourne,450000.0,2.8,Northern Metropolitan,2.0,1.0,55.0,150.504307,918.738956,3407.204651,3367.0
...,...,...,...,...,...,...,...,...,...,...,...,...
13575,3437,Gisborne,650000.0,45.9,Northern Victoria,6.0,2.0,NaN,NaN,NaN,NaN,NaN
13576,3756,Wallan,366000.0,44.2,Northern Victoria,3.0,1.0,77.0,NaN,NaN,NaN,NaN
13577,3022,Ardeer,730000.0,12.8,Western Metropolitan,3.0,2.0,112.0,NaN,NaN,NaN,NaN
13578,3061,Campbellfield,650000.0,16.6,Northern Metropolitan,4.0,2.0,175.0,NaN,NaN,NaN,NaN


## Ejercicio 2 - Pandas:

1. Seleccionar un subconjunto de columnas que les parezcan relevantes al problema de predicción del valor de la propiedad. Justificar las columnas seleccionadas y las que no lo fueron.
 - Eliminar los valores extremos que no sean relevantes para la predicción de valores de las propiedades.


2. Agregar información adicional respectiva al entorno de una propiedad a partir del [conjunto de datos de AirBnB](https://www.kaggle.com/tylerx/melbourne-airbnb-open-data?select=cleansed_listings_dec18.csv) utilizado en el práctico.
  1. Seleccionar qué variables agregar y qué combinaciones aplicar a cada una. Por ejemplo, pueden utilizar solo la columna `price`, o aplicar múltiples transformaciones como la mediana o el mínimo.
  1. Utilizar la variable zipcode para unir los conjuntos de datos. Sólo incluir los zipcodes que tengan una cantidad mínima de registros (a elección) como para que la información agregada sea relevante.
  2. Investigar al menos otras 2 variables que puedan servir para combinar los datos, y justificar si serían adecuadas o no. Pueden asumir que cuentan con la ayuda de anotadores expertos para encontrar equivalencias entre barrios o direcciones, o que cuentan con algoritmos para encontrar las n ubicaciones más cercanas a una propiedad a partir de sus coordenadas geográficas. **NO** es necesario que realicen la implementación.

Pueden leer otras columnas del conjunto de AirBnB además de las que están en `interesting_cols`, si les parecen relevantes.

## Ejercicio 3:

Crear y guardar un nuevo conjunto de datos con todas las transformaciones realizadas anteriormente.

## Ejercicios opcionales:

1. Armar un script en python (archivo .py) [ETL](https://towardsdatascience.com/what-to-log-from-python-etl-pipelines-9e0cfe29950e) que corra los pasos de extraccion, transformacion y carga, armando una funcion para cada etapa del proceso y luego un main que corra todos los pasos requeridos.

2. Armar un DAG en Apache Airflow que corra el ETL. (https://airflow.apache.org/docs/apache-airflow/stable/tutorial.html)